In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sisso import SISSO

## Load Data

In [2]:
K = pickle.load(open("evals_52k.pkl", "rb"))
K.shape

(52008, 3702)

In [3]:
y = pd.read_csv("heusler_data.csv")["band_gap"].values
y.shape

(3702,)

## Set Up Experiment

In [4]:
experiment = SISSO(K=K.T, y=y)

In [5]:
experiment.fit()

INFO:root:------------------
INFO:root:Iteration: 1
INFO:root:Error: 78.19239731575202
INFO:root:Number of combinations: 25
INFO:root:Optimal combination: (4974,)
INFO:root:Optimal coefficients: [65.65757184]


INFO:root:------------------
INFO:root:Iteration: 2
INFO:root:Error: 76.57978837985046
INFO:root:Number of combinations: 1225
INFO:root:Optimal combination: (4833, 36173)
INFO:root:Optimal coefficients: [ 70.82318253 -20.07190652]
INFO:root:------------------
INFO:root:Iteration: 3
INFO:root:Error: 75.27691984028422
INFO:root:Number of combinations: 47905
INFO:root:Optimal combination: (4220, 4462, 36173)
INFO:root:Optimal coefficients: [-38.54778034 -35.82824478 -15.18685514]
INFO:root:------------------
INFO:root:Iteration: 4
INFO:root:Error: 74.30076851277661
INFO:root:Number of combinations: 1663740
INFO:root:Optimal combination: (4220, 4462, 36173, 48297)
INFO:root:Optimal coefficients: [-39.34624565 -36.74307192 -15.5572908   12.18750156]
